In [1]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Add Lincoln to system path
import sys
sys.path.append("/Users/seth/development/lincoln/")

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
from torch import Tensor

import lincoln
from lincoln.layers import Dense
from lincoln.losses import LogSoftmaxLoss, LogSigmoidLoss, MeanSquaredError
from lincoln.optimizers import SGD
from lincoln.operations.activations import Sigmoid, LogSigmoid
from lincoln.network import NeuralNetwork

## Regression network

In [4]:
from sklearn.datasets import load_boston

boston = load_boston()

data = boston.data
target = boston.target
features = boston.feature_names

from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=80718)

X_train, X_test, y_train, y_test = Tensor(X_train), Tensor(X_test), Tensor(y_train), Tensor(y_test)

## Regression models

In [5]:
from lincoln.losses import MeanSquaredError
from lincoln.optimizers import SGD

from lincoln.operations.activations import Sigmoid, LinearAct

#### Linear Regression

In [6]:
from lincoln.train import Trainer

In [7]:
lr = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=LinearAct())], 
            loss = MeanSquaredError())

trainer = Trainer(lr, SGD())

In [8]:
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 10,
       eval_every = 1,
       seed=10818,
       single_output=True);

Validation loss after 1 epochs is 6033.821
Validation loss after 2 epochs is 4218.505
Validation loss after 3 epochs is 4104.863
Validation loss after 4 epochs is 4070.815
Validation loss after 5 epochs is 4214.715
Validation loss after 6 epochs is 4000.082
Validation loss after 7 epochs is 3891.465
Validation loss after 8 epochs is 4202.666
Validation loss after 9 epochs is 3933.137
Validation loss after 10 epochs is 4886.304


In [9]:
def mae(y_true, y_pred):
    return round(torch.mean(torch.abs(y_true - y_pred)).item(), 4)

def mse(y_true, y_pred):
    return round(torch.mean(torch.pow(y_true - y_pred, 2)).item(), 4)

In [1]:
def eval_regression_model(lr: NeuralNetwork, 
                          X_test: Tensor, 
                          y_test: Tensor):
    preds = lr.forward(X_test)
    preds = preds.reshape(preds.shape[0])
    return mae(preds, y_test), mse(preds, y_test)

NameError: name 'NeuralNetwork' is not defined

In [2]:
eval_regression_model(lr, X_test, y_test)

NameError: name 'eval_regression_model' is not defined

Works!

#### Testing momentum

In [12]:
from lincoln.optimizers import SGDMomentum

In [13]:
lr = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=LinearAct())], 
            loss = MeanSquaredError())
trainer = Trainer(lr, SGD(lr=0.001))

trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 50,
       eval_every = 10,
       seed=10818,
       single_output=True);
eval_regression_model(lr, X_test, y_test)

Validation loss after 10 epochs is 4166.400
Validation loss after 20 epochs is 4087.505
Validation loss after 30 epochs is 3822.435
Validation loss after 40 epochs is 3792.870
Validation loss after 50 epochs is 3980.900


(3.6296, 26.1901)

#### "Neural Network" Regression

In [14]:
nn = NeuralNetwork(
    layers=[Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=LinearAct())], 
            loss = MeanSquaredError())

trainer = Trainer(nn, SGD(lr=0.001))
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 50,
       eval_every = 10,
       seed=10818,
       single_output=True);

Validation loss after 10 epochs is 2918.497
Validation loss after 20 epochs is 2351.871
Validation loss after 30 epochs is 2177.422
Validation loss after 40 epochs is 2105.777
Validation loss after 50 epochs is 2126.681


In [15]:
eval_regression_model(nn, X_test, y_test)

(2.3128, 13.9913)

Works!

#### "Deep Learning" Regression

#### Without momentum

In [16]:
dl = NeuralNetwork(
    layers=[Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=7, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=LinearAct())], 
            loss = MeanSquaredError())

trainer = Trainer(dl, SGD(lr=0.001))
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 50,
       eval_every = 10,
       seed=10818,
       single_output=True);

Validation loss after 10 epochs is 2830.598
Validation loss after 20 epochs is 2434.323
Validation loss after 30 epochs is 2282.844
Validation loss after 40 epochs is 1921.820
Validation loss after 50 epochs is 1715.640


In [17]:
eval_regression_model(dl, X_test, y_test)

(2.1504, 11.2871)

#### With momentum

In [18]:
dl = NeuralNetwork(
    layers=[Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=4, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=LinearAct())], 
            loss = MeanSquaredError())

trainer = Trainer(dl, SGDMomentum(lr=0.001,
                                  momentum=0.1))
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 50,
       eval_every = 1,
       seed=10818,
       single_output=True);

Validation loss after 1 epochs is 13593.652
Validation loss after 2 epochs is 10841.587
Validation loss after 3 epochs is 7182.547
Validation loss after 4 epochs is 5510.923
Validation loss after 5 epochs is 4674.278
Validation loss after 6 epochs is 4593.970
Validation loss after 7 epochs is 3753.076
Validation loss after 8 epochs is 3462.191
Validation loss after 9 epochs is 3359.814
Validation loss after 10 epochs is 3106.689
Validation loss after 11 epochs is 3149.418
Validation loss after 12 epochs is 3188.762
Validation loss after 13 epochs is 2995.000
Validation loss after 14 epochs is 3198.239
Validation loss after 15 epochs is 3468.489
Validation loss after 16 epochs is 2873.373
Validation loss after 17 epochs is 3151.231
Validation loss after 18 epochs is 3022.445
Validation loss after 19 epochs is 2965.600
Validation loss after 20 epochs is 2879.256
Validation loss after 21 epochs is 2641.998
Validation loss after 22 epochs is 3111.276
Validation loss after 23 epochs is 2702

In [19]:
eval_regression_model(dl, X_test, y_test)

(2.3293, 13.3238)

In [20]:
dl = NeuralNetwork(
    layers=[Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=LinearAct())], 
            loss = MeanSquaredError())

trainer2 = Trainer(dl, SGDMomentum())
trainer2.fit(X_train, y_train, X_test, y_test,  
       epochs = 10,
       eval_every = 1,
       seed=10818,
       single_output=True);

Validation loss after 1 epochs is 5917.639
Validation loss after 2 epochs is 5294.524
Validation loss after 3 epochs is 6038.587
Validation loss after 4 epochs is 6101.629
Validation loss after 5 epochs is 5729.192
Validation loss after 6 epochs is 6469.512
Validation loss after 7 epochs is 4898.928
Validation loss after 8 epochs is 6898.198
Validation loss after 9 epochs is 6214.041
Validation loss after 10 epochs is 8169.327


In [21]:
eval_regression_model(dl, X_test, y_test)

(5.0019, 53.7456)

Works!

## Classification models

#### Data prep (SciKit Learn)

In [22]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
data = breast_cancer.data
target = breast_cancer.target

In [23]:
from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=82618)

X_train, X_test, y_train, y_test = Tensor(X_train), Tensor(X_test), Tensor(y_train), Tensor(y_test)

#### "`LogSigmoid`" Logistic Regression

In [24]:
logr = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=LogSigmoid())],
            loss = LogSigmoidLoss())

trainer = Trainer(logr, SGD())
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=82618,
       single_output=True);

Validation loss after 10 epochs is 25.874
Validation loss after 20 epochs is 21.193
Validation loss after 30 epochs is 19.611
Validation loss after 40 epochs is 18.724
Validation loss after 50 epochs is 18.189
Validation loss after 60 epochs is 17.802
Validation loss after 70 epochs is 17.536
Validation loss after 80 epochs is 17.347
Validation loss after 90 epochs is 17.219
Validation loss after 100 epochs is 17.125


In [25]:
logr_mse = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = MeanSquaredError())

trainer = Trainer(logr_mse, SGD())
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=82618,
       single_output=True);

Validation loss after 10 epochs is 14.569
Validation loss after 20 epochs is 10.926
Validation loss after 30 epochs is 8.928
Validation loss after 40 epochs is 7.602
Validation loss after 50 epochs is 6.771
Validation loss after 60 epochs is 6.281
Validation loss after 70 epochs is 5.944
Validation loss after 80 epochs is 5.678
Validation loss after 90 epochs is 5.458
Validation loss after 100 epochs is 5.265


In [26]:
def accuracy(y_true, y_pred):
    return round(torch.sum(torch.eq(y_true, y_pred)).item() / y_true.size()[0], 4)

In [27]:
def eval_classification_model(model: NeuralNetwork, 
                              X_test: Tensor, 
                              y_test: Tensor,
                              log_probs: bool = False):
    preds = model.forward(X_test)
    if log_probs:
        preds = torch.exp(preds)
    preds = preds > 0.5
    preds = preds.reshape(preds.shape[0]).type(torch.FloatTensor)  
    return accuracy(preds, y_test)

In [28]:
eval_classification_model(logr, X_test, y_test, log_probs=True)

0.9825

In [29]:
eval_classification_model(logr_mse, X_test, y_test)

0.9766

`LogSoftmaxLoss` works!

In [30]:
logr_mse = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = MeanSquaredError())
trainer_m = Trainer(logr_mse, SGDMomentum())
trainer_m.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=82618,
       single_output=True);

Validation loss after 10 epochs is 5.762
Validation loss after 20 epochs is 3.738
Validation loss after 30 epochs is 3.522
Validation loss after 40 epochs is 3.446
Validation loss after 50 epochs is 3.423
Validation loss after 60 epochs is 3.698
Validation loss after 70 epochs is 3.936
Validation loss after 80 epochs is 4.131
Validation loss after 90 epochs is 4.260
Validation loss after 100 epochs is 4.342


In [31]:
eval_classification_model(logr_mse, X_test, y_test)

0.9708

#### "Neural Network" Logistic Regression

In [32]:
logr_nn = NeuralNetwork(
    layers=[Dense(neurons=30, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = MeanSquaredError())
trainer_m = Trainer(logr_nn, SGDMomentum())
trainer_m.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=10818,
       single_output=True);

Validation loss after 10 epochs is 5.475
Validation loss after 20 epochs is 4.830
Validation loss after 30 epochs is 4.498
Validation loss after 40 epochs is 4.313
Validation loss after 50 epochs is 4.204
Validation loss after 60 epochs is 4.092
Validation loss after 70 epochs is 4.009
Validation loss after 80 epochs is 3.958
Validation loss after 90 epochs is 3.937
Validation loss after 100 epochs is 3.905


In [33]:
eval_classification_model(logr_nn, X_test, y_test)

0.9766

**Works!**